In [1]:
import pandas as pd
import numpy as np
import regress
from sklearn.model_selection import KFold

In [2]:
top_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/fmri'
data_dir = f'{top_dir}/input_data'
out_dir = f'{top_dir}/output_data'
figure_dir = f'{top_dir}/figures'

In [3]:
control_model = np.load(f'{out_dir}/generate_models/control_model.npy')
X = np.load(f'{out_dir}/generate_models/annotated_model.npy')

# Get the feature names for the annotated model
feature_categories = ['visual', 'visual', 'visual', 'visual', 'visual',
             'social detection', 'social detection', 
             'social evaluation', 'social evaluation', 'social evaluation', 'social evaluation', 'social evaluation']
features = pd.read_csv(f'{data_dir}/annotations/annotations.csv').columns.to_list()
features.remove('video_name')

noise_ceiling = pd.read_csv(f'{out_dir}/reliability/noise_ceiling.csv')
noise_ceiling = noise_ceiling.groupby('ROIs').mean().reset_index()

In [ ]:
#Save the rs and null distributions
kf = KFold(n_splits=2, shuffle=True, random_state=1)
sid_ = 0 
sid = str(sid_+1).zfill(2)
beta_map = np.load(f'{out_dir}/grouped_runs/sub-{sid}/sub-{sid}_train-data.npy')
rs = regress.outer_ridge_2d(X, control_model, beta_map.T, len(features), kf)